## Imports

In [33]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [34]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Feb. 10th Feb. 17th


In [35]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [36]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [37]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [38]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [39]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,NaN,Condo/Co-op,4556 N Ocean Dr #1,Lauderdale By The Sea,FL,33308.0,335000.0,2.0,2.0,...,NaN,NaN,NaN,https://www.redfin.com/FL/Lauderdale-by-the-Se...,NaN,NaN,N,Y,26.194431,-80.095969


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [40]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [42]:
df2 = df[(df['SOLD DATE'] > '2025-02-10') & (df['SOLD DATE'] < '2025-02-17')]

In [43]:
df2['SOLD DATE'].value_counts()

SOLD DATE
2025-02-14    25
2025-02-12    11
2025-02-11     7
2025-02-13     7
2025-02-16     1
Name: count, dtype: int64

In [44]:
df2 = df2.sort_values(by='PRICE',ascending=False)

In [69]:
# df2 = df2.drop(index=64)

In [45]:
df_top_ten = df2.head(10)

In [46]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
140,PAST SALE,2025-02-14,Condo/Co-op,525 N Fort Lauderdale Beach Blvd #1501,Fort Lauderdale,FL,33304.0,5750000.0,4.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/525-...,Beaches MLS,F10469173,N,Y,26.130150,-80.104103
182,PAST SALE,2025-02-13,Condo/Co-op,3200 N Ocean Blvd #1809,Fort Lauderdale,FL,33308.0,3200000.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3200...,Beaches MLS,F10458961,N,Y,26.168793,-80.098465
25,PAST SALE,2025-02-14,Condo/Co-op,435 Bayshore Dr #301,Fort Lauderdale,FL,33304.0,2950000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/435-...,Beaches MLS,F10483345,N,Y,26.128681,-80.107989
148,PAST SALE,2025-02-11,Condo/Co-op,1850 S Ocean Dr #4103,Hallandale Beach,FL,33009.0,1425000.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/185...,MARMLS,A11663132,N,Y,25.984216,-80.118868
154,PAST SALE,2025-02-12,Condo/Co-op,411 N New River Dr #405,Fort Lauderdale,FL,33301.0,1200000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/411-...,Beaches MLS,F10465493,N,Y,26.118085,-80.138584
66,PAST SALE,2025-02-14,Condo/Co-op,1 Las Olas Cir #1104,Fort Lauderdale,FL,33316.0,800000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1-La...,Beaches MLS,F10476190,N,Y,26.118076,-80.107100
205,PAST SALE,2025-02-16,Condo/Co-op,1 Las Olas Cir #1117,Fort Lauderdale,FL,33316.0,710000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1-La...,Beaches MLS,F10448763,N,Y,26.118136,-80.107099
201,PAST SALE,2025-02-12,Condo/Co-op,3020 NE 32nd Ave #908,Fort Lauderdale,FL,33308.0,650000.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3020...,MARMLS,A11612252,N,Y,26.166000,-80.103194
172,PAST SALE,2025-02-14,Condo/Co-op,1800 S Ocean Dr #1806,Hallandale Beach,FL,33009.0,645000.0,2.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/180...,MARMLS,A11661975,N,Y,25.985667,-80.118693
63,PAST SALE,2025-02-14,Condo/Co-op,4540 N Ocean Dr #301,Lauderdale By The Sea,FL,33308.0,632500.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderdale-by-the-Se...,Beaches MLS,F10475737,N,Y,26.193843,-80.095982


In [12]:
# df_top_ten = df2.head(10)

## Collect Agent Information

In [47]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [48]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [49]:
response_df = pd.DataFrame(response_list)

In [50]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [51]:
df_top_ten = merged_df

In [52]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2
0,PAST SALE,2025-02-14,Condo/Co-op,525 N Fort Lauderdale Beach Blvd #1501,Fort Lauderdale,FL,33304.0,5750000.0,4.0,4.5,...,-80.104103,https://www.redfin.com/FL/Fort-Lauderdale/525-...,Daniel Teixeira,Douglas Elliman,Enrique Teixeira,Douglas Elliman,Daniel Teixeira,Douglas Elliman,None,None
1,PAST SALE,2025-02-13,Condo/Co-op,3200 N Ocean Blvd #1809,Fort Lauderdale,FL,33308.0,3200000.0,3.0,3.0,...,-80.098465,https://www.redfin.com/FL/Fort-Lauderdale/3200...,Patricia Post,"Post and Post, Inc.",Patricia Post,"Post and Post, Inc.",Andrew Nowacki,LoKation,None,None
2,PAST SALE,2025-02-14,Condo/Co-op,435 Bayshore Dr #301,Fort Lauderdale,FL,33304.0,2950000.0,3.0,3.5,...,-80.107989,https://www.redfin.com/FL/Fort-Lauderdale/435-...,Tika Van Den Hurk,Douglas Elliman,None,None,Jennifer Gitlan,Coldwell Banker Realty,Jennifer Gitlan,Coldwell Banker Realty
3,PAST SALE,2025-02-11,Condo/Co-op,1850 S Ocean Dr #4103,Hallandale Beach,FL,33009.0,1425000.0,3.0,3.0,...,-80.118868,https://www.redfin.com/FL/Hallandale-Beach/185...,Juliana Tobon,Cervera Real Estate Inc,None,None,Maria Bustamante,Cervera Real Estate Inc,None,None
4,PAST SALE,2025-02-12,Condo/Co-op,411 N New River Dr #405,Fort Lauderdale,FL,33301.0,1200000.0,2.0,2.0,...,-80.138584,https://www.redfin.com/FL/Fort-Lauderdale/411-...,John D'Angelo,D'Angelo Realty Group LLC,None,None,Carlos Martin,Good People Realty,None,None
5,PAST SALE,2025-02-14,Condo/Co-op,1 Las Olas Cir #1104,Fort Lauderdale,FL,33316.0,800000.0,2.0,2.0,...,-80.107100,https://www.redfin.com/FL/Fort-Lauderdale/1-La...,Gilmar Scantamburlo,United Realty Group Inc,None,None,Gilmar Scantamburlo,United Realty Group Inc,None,None
6,PAST SALE,2025-02-16,Condo/Co-op,1 Las Olas Cir #1117,Fort Lauderdale,FL,33316.0,710000.0,2.0,2.0,...,-80.107099,https://www.redfin.com/FL/Fort-Lauderdale/1-La...,Gilmar Scantamburlo,United Realty Group Inc,None,None,Gilmar Scantamburlo,United Realty Group Inc,None,None
7,PAST SALE,2025-02-12,Condo/Co-op,3020 NE 32nd Ave #908,Fort Lauderdale,FL,33308.0,650000.0,3.0,2.0,...,-80.103194,https://www.redfin.com/FL/Fort-Lauderdale/3020...,Cynthia Wolf,Galleria International Realty,None,None,Cynthia Wolf,Galleria International Realty,None,None
8,PAST SALE,2025-02-14,Condo/Co-op,1800 S Ocean Dr #1806,Hallandale Beach,FL,33009.0,645000.0,2.0,1.0,...,-80.118693,https://www.redfin.com/FL/Hallandale-Beach/180...,Perla Garcia,Fortune Christie's Intl R.E.,Andrea Langesfeld,Fortune Christie's Intl R.E.,Lana Levitina,Coldwell Banker Realty,None,None
9,PAST SALE,2025-02-14,Condo/Co-op,4540 N Ocean Dr #301,Lauderdale By The Sea,FL,33308.0,632500.0,2.0,2.0,...,-80.095982,https://www.redfin.com/FL/Lauderdale-by-the-Se...,Susan Gauthier,"By The Sea Realty, Inc.",None,None,Susan Gauthier,"By The Sea Realty, Inc.",None,None


## Current Week's Values

In [53]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

51


In [54]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$625,196


In [55]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$428


In [56]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$31,885,001


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [22]:
# # Define the date range
# start_date = '2024-12-23'
# end_date = '2024-12-29'

# # Filter the DataFrame between two dates (inclusive)
# old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [57]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

## Clean Data

Input county name:
Broward County
Input Previous Week Condo Sales Total (number units sold):
104
Input Previous Week Condo Average Sales Price:
340736
Input Previous Week Condo Average PSF:
270
Input Previous Week Condo Sales total (ex: 198_000_000)
35_400_000


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [58]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [59]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [60]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [61]:
pd.set_option('display.max_columns',None)

In [62]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-02-14,Condo/Co-op,525 N Fort Lauderdale Beach Blvd #1501,Fort Lauderdale,FL,33304.0,5750000.0,4.0,4.5,Four Seasons,2575.0,NaN,2022.0,NaN,2233.0,5298.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/525-...,Beaches MLS,F10469173,N,Y,26.13015,-80.104103,https://www.redfin.com/FL/Fort-Lauderdale/525-...,Daniel Teixeira,Douglas Elliman,Enrique Teixeira,Douglas Elliman,Daniel Teixeira,Douglas Elliman,None,None,1,orange


In [63]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [64]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-02-14,Condo/Co-op,525 N Fort Lauderdale Beach Blvd #1501,Fort Lauderdale,FL,33304.0,5750000.0,4.0,4.5,Four Seasons,2575.0,NaN,2022.0,NaN,2233.0,5298.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/525-...,Beaches MLS,F10469173,N,Y,26.130150,-80.104103,https://www.redfin.com/FL/Fort-Lauderdale/525-...,Daniel Teixeira,Douglas Elliman,Enrique Teixeira,Douglas Elliman,Daniel Teixeira,Douglas Elliman,None,None,1,orange
1,PAST SALE,2025-02-13,Condo/Co-op,3200 N Ocean Blvd #1809,Fort Lauderdale,FL,33308.0,3200000.0,3.0,3.0,L'hermitage,2845.0,NaN,1998.0,NaN,1125.0,3956.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3200...,Beaches MLS,F10458961,N,Y,26.168793,-80.098465,https://www.redfin.com/FL/Fort-Lauderdale/3200...,Patricia Post,"Post and Post, Inc.",Patricia Post,"Post and Post, Inc.",Andrew Nowacki,LoKation,None,None,2,blue
2,PAST SALE,2025-02-14,Condo/Co-op,435 Bayshore Dr #301,Fort Lauderdale,FL,33304.0,2950000.0,3.0,3.5,Adagio,2383.0,NaN,2019.0,NaN,1238.0,3694.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/435-...,Beaches MLS,F10483345,N,Y,26.128681,-80.107989,https://www.redfin.com/FL/Fort-Lauderdale/435-...,Tika Van Den Hurk,Douglas Elliman,None,None,Jennifer Gitlan,Coldwell Banker Realty,Jennifer Gitlan,Coldwell Banker Realty,3,blue
3,PAST SALE,2025-02-11,Condo/Co-op,1850 S Ocean Dr #4103,Hallandale Beach,FL,33009.0,1425000.0,3.0,3.0,BEACH CLUB CONDO,1841.0,NaN,2005.0,NaN,774.0,1954.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/185...,MARMLS,A11663132,N,Y,25.984216,-80.118868,https://www.redfin.com/FL/Hallandale-Beach/185...,Juliana Tobon,Cervera Real Estate Inc,None,None,Maria Bustamante,Cervera Real Estate Inc,None,None,4,blue
4,PAST SALE,2025-02-12,Condo/Co-op,411 N New River Dr #405,Fort Lauderdale,FL,33301.0,1200000.0,2.0,2.0,Las Olas Grand,1630.0,NaN,2005.0,NaN,736.0,2080.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/411-...,Beaches MLS,F10465493,N,Y,26.118085,-80.138584,https://www.redfin.com/FL/Fort-Lauderdale/411-...,John D'Angelo,D'Angelo Realty Group LLC,None,None,Carlos Martin,Good People Realty,None,None,5,blue
5,PAST SALE,2025-02-14,Condo/Co-op,1 Las Olas Cir #1104,Fort Lauderdale,FL,33316.0,800000.0,2.0,2.0,Venetian Condominium,1500.0,NaN,1971.0,NaN,533.0,1286.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1-La...,Beaches MLS,F10476190,N,Y,26.118076,-80.107100,https://www.redfin.com/FL/Fort-Lauderdale/1-La...,Gilmar Scantamburlo,United Realty Group Inc,None,None,Gilmar Scantamburlo,United Realty Group Inc,None,None,6,blue
6,PAST SALE,2025-02-16,Condo/Co-op,1 Las Olas Cir #1117,Fort Lauderdale,FL,33316.0,710000.0,2.0,2.0,Venetian Condominium,1350.0,NaN,1971.0,NaN,526.0,1199.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1-La...,Beaches MLS,F10448763,N,Y,26.118136,-80.107099,https://www.redfin.com/FL/Fort-Lauderdale/1-La...,Gilmar Scantamburlo,United Realty Group Inc,None,None,Gilmar Scantamburlo,United Realty Group Inc,None,None,7,blue
7,PAST SALE,2025-02-12,Condo/Co-op,3020 NE 32nd Ave #908,Fort Lauderdale,FL,33308.0,650000.0,3.0,2.0,TIDES AT BRIDGESIDE SQUAR,1600.0,NaN,2002.0,NaN,406.0,1891.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3020...,MARMLS,A11612252,N,Y,26.166000,-80.103194,https://www.redfin.com/FL/Fort-Lauderdale/3020...,Cynthia Wolf,Galleria International Realty,None,None,Cynthia Wolf,Galleria International Realty,None,None,8,blue
8,PAST SALE,2025-02-14,Condo/Co-op,1800 S Ocean Dr #1806,Hallandale Beach,FL,33009.0,645000.0,2.0,1.0,Beach Club Three Co

In [65]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent {county} County Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} — {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [66]:
m.save('index.html')

## Data snagger

In [67]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [68]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-02-14,Condo/Co-op,525 N Fort Lauderdale Beach Blvd #1501,Fort Lauderdale,FL,33304.0,5750000.0,4.0,4.5,Four Seasons,2575.0,NaN,2022.0,NaN,2233.0,5298.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/525-...,Beaches MLS,F10469173,N,Y,26.13015,-80.104103,https://www.redfin.com/FL/Fort-Lauderdale/525-...,Daniel Teixeira,Douglas Elliman,Enrique Teixeira,Douglas Elliman,Daniel Teixeira,Douglas Elliman,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [69]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/Broward_County_condo_sales_week_ending_02182024


# CREATE TEMPLATE 

In [70]:
muni_set = set(df_top_ten['CITY'])

In [71]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [72]:
df_top_ten.reset_index(inplace=True,drop=True)

In [73]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [74]:
top_sale

'Four Seasons at 525 N Fort Lauderdale Beach Blvd #1501 in Fort Lauderdale'

In [75]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [76]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [77]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-02-14,Condo/Co-op,525 N Fort Lauderdale Beach Blvd #1501,Fort Lauderdale,FL,33304.0,5750000.0,4.0,4.5,Four Seasons,2575.0,NaN,2022.0,NaN,2233.0,5298.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/525-...,Beaches MLS,F10469173,N,Y,26.130150,-80.104103,https://www.redfin.com/FL/Fort-Lauderdale/525-...,Daniel Teixeira,Douglas Elliman,Enrique Teixeira,Douglas Elliman,Daniel Teixeira,Douglas Elliman,None,None,1,orange,Four Seasons at 525 N Fort Lauderdale Beach Bl...
1,PAST SALE,2025-02-13,Condo/Co-op,3200 N Ocean Blvd #1809,Fort Lauderdale,FL,33308.0,3200000.0,3.0,3.0,L'hermitage,2845.0,NaN,1998.0,NaN,1125.0,3956.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3200...,Beaches MLS,F10458961,N,Y,26.168793,-80.098465,https://www.redfin.com/FL/Fort-Lauderdale/3200...,Patricia Post,"Post and Post, Inc.",Patricia Post,"Post and Post, Inc.",Andrew Nowacki,LoKation,None,None,2,blue,L'hermitage at 3200 N Ocean Blvd #1809 in Fort...
2,PAST SALE,2025-02-14,Condo/Co-op,435 Bayshore Dr #301,Fort Lauderdale,FL,33304.0,2950000.0,3.0,3.5,Adagio,2383.0,NaN,2019.0,NaN,1238.0,3694.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/435-...,Beaches MLS,F10483345,N,Y,26.128681,-80.107989,https://www.redfin.com/FL/Fort-Lauderdale/435-...,Tika Van Den Hurk,Douglas Elliman,None,None,Jennifer Gitlan,Coldwell Banker Realty,Jennifer Gitlan,Coldwell Banker Realty,3,blue,Adagio at 435 Bayshore Dr #301 in Fort Lauderdale
3,PAST SALE,2025-02-11,Condo/Co-op,1850 S Ocean Dr #4103,Hallandale Beach,FL,33009.0,1425000.0,3.0,3.0,BEACH CLUB CONDO,1841.0,NaN,2005.0,NaN,774.0,1954.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/185...,MARMLS,A11663132,N,Y,25.984216,-80.118868,https://www.redfin.com/FL/Hallandale-Beach/185...,Juliana Tobon,Cervera Real Estate Inc,None,None,Maria Bustamante,Cervera Real Estate Inc,None,None,4,blue,BEACH CLUB CONDO at 1850 S Ocean Dr #4103 in H...
4,PAST SALE,2025-02-12,Condo/Co-op,411 N New River Dr #405,Fort Lauderdale,FL,33301.0,1200000.0,2.0,2.0,Las Olas Grand,1630.0,NaN,2005.0,NaN,736.0,2080.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/411-...,Beaches MLS,F10465493,N,Y,26.118085,-80.138584,https://www.redfin.com/FL/Fort-Lauderdale/411-...,John D'Angelo,D'Angelo Realty Group LLC,None,None,Carlos Martin,Good People Realty,None,None,5,blue,Las Olas Grand at 411 N New River Dr #405 in F...
5,PAST SALE,2025-02-14,Condo/Co-op,1 Las Olas Cir #1104,Fort Lauderdale,FL,33316.0,800000.0,2.0,2.0,Venetian Condominium,1500.0,NaN,1971.0,NaN,533.0,1286.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1-La...,Beaches MLS,F10476190,N,Y,26.118076,-80.107100,https://www.redfin.com/FL/Fort-Lauderdale/1-La...,Gilmar Scantamburlo,United Realty Group Inc,None,None,Gilmar Scantamburlo,United Realty Group Inc,None,None,6,blue,Venetian Condominium at 1 Las Olas Cir #1104 i...
6,PAST SALE,2025-02-16,Condo/Co-op,1 Las Olas Cir #1117,Fort Lauderdale,FL,33316.0,710000.0,2.0,2.0,Venetian Condominium,1350.0,NaN,1971.0,NaN,526.0,1199.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1-La...,Beaches MLS,F10448763,N,Y,26.118136,-80.107099,https://www.redfin.com/FL/Fort-Lauderdale/1-La...,Gilmar Scantamburlo,United Realty Group Inc,None,None,Gilmar Scantamburlo,United Realty Group Inc,None,None,7,blue,Venetian Condominium at 1 Las Olas Cir #1117 i...
7,PAST SALE,2025-02-12,Condo/Co-op,3020 NE 32nd Ave #908,Fort Lauderdale,FL,33308.0,650000.0,3.0,2.0,TIDES AT BRIDGESIDE SQUAR,1600.0,NaN,2002.0,NaN,406.0,1891.0,Sold,NaN,NaN,https://www.redf

In [78]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: Four Seasons closing tops Broward County weekly condo sales 
DEK: Top 10 sales ranged from $632,500 to $5,750,000
FEATURED HED:
SEO HED: Broward County County Weekly Condo Report 
SEO DESCRIPTION: Broward County County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Broward County condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Broward County, weekly condo sales, Lauderdale By The Sea, Fort Lauderdale, Hallandale Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Broward County County’s NEWS PEG HERE.

Brokers closed 51 condo sales totaling $31,885,001 million from Feb. 10th to Feb. 17th. The previous week, brokers closed 104 condo sales totaling $35,400,000.

Last week’s units sold for an average of $625,196, higher than the $340,736 average price from the pr

In [79]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [80]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [81]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [82]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-02-14,Condo/Co-op,525 N Fort Lauderdale Beach Blvd #1501,Fort Lauderdale,FL,33304.0,5750000.0,4.0,4.5,Four Seasons,2575.0,NaN,2022.0,NaN,2233.0,5298.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/525-...,Beaches MLS,F10469173,N,Y,26.130150,-80.104103,https://www.redfin.com/FL/Fort-Lauderdale/525-...,Daniel Teixeira,Douglas Elliman,Enrique Teixeira,Douglas Elliman,Daniel Teixeira,Douglas Elliman,None,None,1,orange,Four Seasons at 525 N Fort Lauderdale Beach Bl...
1,PAST SALE,2025-02-13,Condo/Co-op,3200 N Ocean Blvd #1809,Fort Lauderdale,FL,33308.0,3200000.0,3.0,3.0,L'hermitage,2845.0,NaN,1998.0,NaN,1125.0,3956.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3200...,Beaches MLS,F10458961,N,Y,26.168793,-80.098465,https://www.redfin.com/FL/Fort-Lauderdale/3200...,Patricia Post,"Post and Post, Inc.",Patricia Post,"Post and Post, Inc.",Andrew Nowacki,LoKation,None,None,2,blue,L'hermitage at 3200 N Ocean Blvd #1809 in Fort...
2,PAST SALE,2025-02-14,Condo/Co-op,435 Bayshore Dr #301,Fort Lauderdale,FL,33304.0,2950000.0,3.0,3.5,Adagio,2383.0,NaN,2019.0,NaN,1238.0,3694.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/435-...,Beaches MLS,F10483345,N,Y,26.128681,-80.107989,https://www.redfin.com/FL/Fort-Lauderdale/435-...,Tika Van Den Hurk,Douglas Elliman,None,None,Jennifer Gitlan,Coldwell Banker Realty,Jennifer Gitlan,Coldwell Banker Realty,3,blue,Adagio at 435 Bayshore Dr #301 in Fort Lauderdale
3,PAST SALE,2025-02-11,Condo/Co-op,1850 S Ocean Dr #4103,Hallandale Beach,FL,33009.0,1425000.0,3.0,3.0,BEACH CLUB CONDO,1841.0,NaN,2005.0,NaN,774.0,1954.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/185...,MARMLS,A11663132,N,Y,25.984216,-80.118868,https://www.redfin.com/FL/Hallandale-Beach/185...,Juliana Tobon,Cervera Real Estate Inc,None,None,Maria Bustamante,Cervera Real Estate Inc,None,None,4,blue,BEACH CLUB CONDO at 1850 S Ocean Dr #4103 in H...
4,PAST SALE,2025-02-12,Condo/Co-op,411 N New River Dr #405,Fort Lauderdale,FL,33301.0,1200000.0,2.0,2.0,Las Olas Grand,1630.0,NaN,2005.0,NaN,736.0,2080.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/411-...,Beaches MLS,F10465493,N,Y,26.118085,-80.138584,https://www.redfin.com/FL/Fort-Lauderdale/411-...,John D'Angelo,D'Angelo Realty Group LLC,None,None,Carlos Martin,Good People Realty,None,None,5,blue,Las Olas Grand at 411 N New River Dr #405 in F...
5,PAST SALE,2025-02-14,Condo/Co-op,1 Las Olas Cir #1104,Fort Lauderdale,FL,33316.0,800000.0,2.0,2.0,Venetian Condominium,1500.0,NaN,1971.0,NaN,533.0,1286.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1-La...,Beaches MLS,F10476190,N,Y,26.118076,-80.107100,https://www.redfin.com/FL/Fort-Lauderdale/1-La...,Gilmar Scantamburlo,United Realty Group Inc,None,None,Gilmar Scantamburlo,United Realty Group Inc,None,None,6,blue,Venetian Condominium at 1 Las Olas Cir #1104 i...
6,PAST SALE,2025-02-16,Condo/Co-op,1 Las Olas Cir #1117,Fort Lauderdale,FL,33316.0,710000.0,2.0,2.0,Venetian Condominium,1350.0,NaN,1971.0,NaN,526.0,1199.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1-La...,Beaches MLS,F10448763,N,Y,26.118136,-80.107099,https://www.redfin.com/FL/Fort-Lauderdale/1-La...,Gilmar Scantamburlo,United Realty Group Inc,None,None,Gilmar Scantamburlo,United Realty Group Inc,None,None,7,blue,Venetian Condominium at 1 Las Olas Cir #1117 i...
7,PAST SALE,2025-02-12,Condo/Co-op,3020 NE 32nd Ave #908,Fort Lauderdale,FL,33308.0,650000.0,3.0,2.0,TIDES AT BRIDGESIDE SQUAR,1600.0,NaN,2002.0,NaN,406.0,1891.0,Sold,NaN,NaN,https://www.redf

In [83]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: Four Seasons closing tops Broward County weekly condo sales 
DEK: Top 10 sales ranged from $632,500 to $5,750,000
FEATURED HED:
SEO HED: Broward County County Weekly Condo Report 
SEO DESCRIPTION: Broward County County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Broward County condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Broward County, weekly condo sales, Lauderdale By The Sea, Fort Lauderdale, Hallandale Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Broward County County’s NEWS PEG HERE.

Brokers closed 51 condo sales totaling $31,885,001 million from Feb. 10th to Feb. 17th. The previous week, brokers closed 104 condo sales totaling $35,400,000.

Last week’s units sold for an average of $625,196, higher than the $340,736 average price from the pr

In [91]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Fort-Lauderdale/525-N-Fort-Lauderdale-Beach-Blvd-33304/unit-1501/home/130656205


In [92]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Fort-Lauderdale/3200-N-Ocean-Blvd-33308/unit-1809/home/41978788


In [93]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Lauderdale-by-the-Sea/4540-N-Ocean-Dr-33308/unit-301/home/41707215


In [94]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Hallandale-Beach/1800-S-Ocean-Dr-33009/unit-1806/home/42073863


In [95]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Fort-Lauderdale/525-N-Fort-Lauderdale-Beach-Blvd-33304/unit-1501/home/130656205


In [96]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Fort-Lauderdale/3020-NE-32nd-Ave-33308/unit-908/home/42022762


In [97]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-02-14,Condo/Co-op,525 N Fort Lauderdale Beach Blvd #1501,Fort Lauderdale,FL,33304.0,5750000.0,4.0,4.5,Four Seasons,2575.0,NaN,2022.0,NaN,2233.0,5298.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/525-...,Beaches MLS,F10469173,N,Y,26.130150,-80.104103,https://www.redfin.com/FL/Fort-Lauderdale/525-...,Daniel Teixeira,Douglas Elliman,Enrique Teixeira,Douglas Elliman,Daniel Teixeira,Douglas Elliman,None,None,1,orange,Four Seasons at 525 N Fort Lauderdale Beach Bl...
1,PAST SALE,2025-02-13,Condo/Co-op,3200 N Ocean Blvd #1809,Fort Lauderdale,FL,33308.0,3200000.0,3.0,3.0,L'hermitage,2845.0,NaN,1998.0,NaN,1125.0,3956.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3200...,Beaches MLS,F10458961,N,Y,26.168793,-80.098465,https://www.redfin.com/FL/Fort-Lauderdale/3200...,Patricia Post,"Post and Post, Inc.",Patricia Post,"Post and Post, Inc.",Andrew Nowacki,LoKation,None,None,2,blue,L'hermitage at 3200 N Ocean Blvd #1809 in Fort...
2,PAST SALE,2025-02-14,Condo/Co-op,435 Bayshore Dr #301,Fort Lauderdale,FL,33304.0,2950000.0,3.0,3.5,Adagio,2383.0,NaN,2019.0,NaN,1238.0,3694.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/435-...,Beaches MLS,F10483345,N,Y,26.128681,-80.107989,https://www.redfin.com/FL/Fort-Lauderdale/435-...,Tika Van Den Hurk,Douglas Elliman,None,None,Jennifer Gitlan,Coldwell Banker Realty,Jennifer Gitlan,Coldwell Banker Realty,3,blue,Adagio at 435 Bayshore Dr #301 in Fort Lauderdale
3,PAST SALE,2025-02-11,Condo/Co-op,1850 S Ocean Dr #4103,Hallandale Beach,FL,33009.0,1425000.0,3.0,3.0,BEACH CLUB CONDO,1841.0,NaN,2005.0,NaN,774.0,1954.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/185...,MARMLS,A11663132,N,Y,25.984216,-80.118868,https://www.redfin.com/FL/Hallandale-Beach/185...,Juliana Tobon,Cervera Real Estate Inc,None,None,Maria Bustamante,Cervera Real Estate Inc,None,None,4,blue,BEACH CLUB CONDO at 1850 S Ocean Dr #4103 in H...
4,PAST SALE,2025-02-12,Condo/Co-op,411 N New River Dr #405,Fort Lauderdale,FL,33301.0,1200000.0,2.0,2.0,Las Olas Grand,1630.0,NaN,2005.0,NaN,736.0,2080.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/411-...,Beaches MLS,F10465493,N,Y,26.118085,-80.138584,https://www.redfin.com/FL/Fort-Lauderdale/411-...,John D'Angelo,D'Angelo Realty Group LLC,None,None,Carlos Martin,Good People Realty,None,None,5,blue,Las Olas Grand at 411 N New River Dr #405 in F...
5,PAST SALE,2025-02-14,Condo/Co-op,1 Las Olas Cir #1104,Fort Lauderdale,FL,33316.0,800000.0,2.0,2.0,Venetian Condominium,1500.0,NaN,1971.0,NaN,533.0,1286.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1-La...,Beaches MLS,F10476190,N,Y,26.118076,-80.107100,https://www.redfin.com/FL/Fort-Lauderdale/1-La...,Gilmar Scantamburlo,United Realty Group Inc,None,None,Gilmar Scantamburlo,United Realty Group Inc,None,None,6,blue,Venetian Condominium at 1 Las Olas Cir #1104 i...
6,PAST SALE,2025-02-16,Condo/Co-op,1 Las Olas Cir #1117,Fort Lauderdale,FL,33316.0,710000.0,2.0,2.0,Venetian Condominium,1350.0,NaN,1971.0,NaN,526.0,1199.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1-La...,Beaches MLS,F10448763,N,Y,26.118136,-80.107099,https://www.redfin.com/FL/Fort-Lauderdale/1-La...,Gilmar Scantamburlo,United Realty Group Inc,None,None,Gilmar Scantamburlo,United Realty Group Inc,None,None,7,blue,Venetian Condominium at 1 Las Olas Cir #1117 i...
7,PAST SALE,2025-02-12,Condo/Co-op,3020 NE 32nd Ave #908,Fort Lauderdale,FL,33308.0,650000.0,3.0,2.0,TIDES AT BRIDGESIDE SQUAR,1600.0,NaN,2002.0,NaN,406.0,1891.0,Sold,NaN,NaN,https://www.redf

## Time on Market Calculator

In [99]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 8, 30) ## List (Earlier) date
date2 = datetime(2025, 2, 14) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

168


## Clean CSV for Datawrapper Chart

In [100]:
chart_df = df_top_ten

In [101]:
chart_df = chart_df.fillna(" ")

In [102]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [103]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [104]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [105]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [106]:
chart_df['$/SQUARE FEET'].astype(float)

0    2233.0
1    1125.0
2    1238.0
3     774.0
4     736.0
5     533.0
6     526.0
7     406.0
8     594.0
9     436.0
Name: $/SQUARE FEET, dtype: float64

In [107]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x) if pd.notnull(x) else x)
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [108]:
csv_date_string = today.strftime("%m_%d_%Y")

In [109]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv",index=False)

In [110]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2025-02-14,525 N Fort Lauderdale Beach Blvd #1501 in Fort...,"$5,750,000",4,4.5,"2,575",2022,"$2,233",Beaches MLS,F10469173,Daniel Teixeira Douglas Elliman Enrique Teixei...,Daniel Teixeira Douglas Elliman
1,2025-02-13,3200 N Ocean Blvd #1809 in Fort Lauderdale,"$3,200,000",3,3.0,"2,845",1998,"$1,125",Beaches MLS,F10458961,"Patricia Post Post and Post, Inc. Patricia Pos...",Andrew Nowacki LoKation
2,2025-02-14,435 Bayshore Dr #301 in Fort Lauderdale,"$2,950,000",3,3.5,"2,383",2019,"$1,238",Beaches MLS,F10483345,Tika Van Den Hurk Douglas Elliman,Jennifer Gitlan Coldwell Banker Realty Jennife...
3,2025-02-11,1850 S Ocean Dr #4103 in Hallandale Beach,"$1,425,000",3,3.0,"1,841",2005,$774,MARMLS,A11663132,Juliana Tobon Cervera Real Estate Inc,Maria Bustamante Cervera Real Estate Inc
4,2025-02-12,411 N New River Dr #405 in Fort Lauderdale,"$1,200,000",2,2.0,"1,630",2005,$736,Beaches MLS,F10465493,John D'Angelo D'Angelo Realty Group LLC,Carlos Martin Good People Realty
5,2025-02-14,1 Las Olas Cir #1104 in Fort Lauderdale,"$800,000",2,2.0,"1,500",1971,$533,Beaches MLS,F10476190,Gilmar Scantamburlo United Realty Group Inc,Gilmar Scantamburlo United Realty Group Inc
6,2025-02-16,1 Las Olas Cir #1117 in Fort Lauderdale,"$710,000",2,2.0,"1,350",1971,$526,Beaches MLS,F10448763,Gilmar Scantamburlo United Realty Group Inc,Gilmar Scantamburlo United Realty Group Inc
7,2025-02-12,3020 NE 32nd Ave #908 in Fort Lauderdale,"$650,000",3,2.0,"1,600",2002,$406,MARMLS,A11612252,Cynthia Wolf Galleria International Realty,Cynthia Wolf Galleria International Realty
8,2025-02-14,1800 S Ocean Dr #1806 in Hallandale Beach,"$645,000",2,1.0,"1,086",2006,$594,MARMLS,A11661975,Perla Garcia Fortune Christie's Intl R.E. Andr...,Lana Levitina Coldwell Banker Realty
9,2025-02-14,4540 N Ocean Dr #301 in Lauderdale By The Sea,"$632,500",2,2.0,"1,450",1970,$436,Beaches MLS,F10475737,"Susan Gauthier By The Sea Realty, Inc.","Susan Gauthier By The Sea Realty, Inc."
